In [1]:
from sklearn.linear_model import LogisticRegression
import joblib

from google.colab import drive
drive.mount('/content/drive')

loaded_model = joblib.load('/content/drive/MyDrive/logistic_model.pkl')

Mounted at /content/drive


In [2]:
import glob

test_data_path = "/content/drive/MyDrive/AI_Project_14/MP4_GOOD/text_results/"

file_list = []
file_contents = []

for file in glob.glob(test_data_path + "/*.txt"):
    file_list.append(file)
    with open(file, 'r') as f:
        content = f.read()
        file_contents.append(content)

In [3]:
print(len(file_list))

89


In [4]:
from gensim.parsing.preprocessing import preprocess_string

pre_docs = []
n_docs = []

for content in file_contents:
    # Preprocess the script text using the gensim preprocessing function
    pre_docs.append(" ".join(preprocess_string(content)))

    # Alternatively, if you don't want to preprocess the script text, you can use the following lines:
    n_docs.append(content)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# Initialize CountVectorizer
vect = CountVectorizer()  # You can specify the vocabulary using 'vocabulary=vocab' or leave it unspecified

# Transform the document collection into a document-term matrix
dtm = vect.fit_transform(pre_docs).toarray()

# Get the vocabulary (feature names)
vocab = vect.get_feature_names_out()

# Initialize TfidfVectorizer with specified vocabulary
tfidfv = TfidfVectorizer(vocabulary=vocab)

# Transform the document collection into a TF-IDF matrix
tfidf = tfidfv.fit_transform(pre_docs).toarray()

# Get the feature names (vocabulary)
vocab = tfidfv.get_feature_names_out()

# Create an array of the indices that would sort each row of the TF-IDF matrix in descending order
tfidf_order = np.flip(np.argsort(tfidf, axis=-1), axis=-1)

topics = []  # List to store the top terms for each document
n_top = 5  # Number of top terms to retrieve for each document

# Iterate over each row in tfidf_order
for line in tfidf_order:
    # Retrieve the top n_top terms for the current document
    topic = [vocab[x] for x in line[:n_top]]
    topics.append(topic)

# Create a pandas DataFrame to display the top terms for each document
df = pd.DataFrame(topics)

In [6]:
%%capture
!pip install transformers
from transformers import pipeline

classifier = pipeline('zero-shot-classification', model='roberta-large-mnli')

In [7]:
a = []  # List to store the indices
b = []  # List to store the scores
c = []  # List to store the counts

for idx, (_) in enumerate(n_docs):
    sequence_to_classify = n_docs[idx]
    candidate_labels = ' '.join(df.iloc[idx][:3])
    score = classifier(sequence_to_classify, candidate_labels)['scores'][0]

    words = sequence_to_classify.split()
    word_count = len(words)

    idx += 1

    a.append(idx)
    b.append(score)
    c.append(word_count)

In [8]:
import csv
import os

test_data = list(zip(a, b, c))
test_data_path = '/content/drive/MyDrive/my_test_data.csv'

if os.path.exists(test_data_path):
    os.remove(test_data_path)

with open(test_data_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['idx', 'score', 'count'])  # 첫 번째 행에 열 이름 쓰기
    writer.writerows(test_data)  # 데이터 쓰기

In [9]:
tdata = pd.read_csv(test_data_path)

X = tdata[['score', 'count']]  # vector?
print(X)

pred = loaded_model.predict(X)  # result
print("Prediction:", pred)

       score  count
0   0.592899   3004
1   0.804751   1530
2   0.530126   1309
3   0.232205   1569
4   0.476543   1834
..       ...    ...
84  0.443273    815
85  0.473831   2715
86  0.413845    886
87  0.792928   1892
88  0.776828   2132

[89 rows x 2 columns]
Prediction: [1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
